In [2]:
import cobra
import pandas as pd
import sys

source_dir = '../src'
sys.path.append(source_dir)
from utils import add_column_to_13c_flux_df
from flux_prediction_scatterplot import flux_prediction_scatterplot
from t_to_f_functions import get_eflux2_solution

### Load genome scale model

In [3]:
model = cobra.io.json.load_json_model("../genome_scale_models/iYLI647_corr.json")
model

Name,model
Memory address,10fd67f40
Number of metabolites,1121
Number of reactions,1348
Number of genes,646
Number of groups,0
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"c, e, m, n, x, r, g, v"


### Load transcriptomics data

In [4]:
# load this file data/transcriptomics/fpkm_counts.csv
transcriptomics_df = pd.read_csv("../data/transcriptomics/fpkm_counts.csv")
transcriptomics_df

,JGI_id,YALI0_id,glucose_1,glucose_2,glucose_3,glycerol_1,glycerol_2,glycerol_3,oleic_acid_1,oleic_acid_2,oleic_acid_3
0,jgi.p|Yarli1|64471,YALI0A00110g,4.88,2.64,3.23,1.21,1.11,3.61,6.77,13.32,7.67
1,jgi.p|Yarli1|64472,YALI0A00132g,1896.43,1845.77,1682.56,1234.99,1189.24,1120.35,864.64,1056.63,928.51
2,jgi.p|Yarli1|64473,YALI0A00154g,31.34,29.21,21.92,215.23,137.85,182.81,1867.59,2153.21,1958.68
3,jgi.p|Yarli1|64474,YALI0A00176g,3.67,3.85,3.15,3.07,1.74,2.09,4.13,2.43,4.34
4,jgi.p|Yarli1|64475,YALI0A00198g,0.00,0.17,0.00,0.00,0.50,1.29,1.55,0.81,1.75
...,...,...,...,...,...,...,...,...,...,...,...
6442,jgi.p|Yarli1|70913,YALI0F32043g,20.63,17.60,17.12,13.94,13.84,11.94,10.76,12.38,10.37
6443,jgi.p|Yarli1|70914,YALI0F32065g,192.58,151.54,270.76,206.89,262.66,280.82,310.55,384.22,313.24
6444,jgi.p|Yarli1|70915,YALI0F32131g,0.12,0.66,0.14,0.34,0.29,0.64,1.01,0.58,1.33
6445,jgi.p|Yarli1|70916,YALI0F32153g,26.16,27.73,28.77,39.22,36.95,39.18,26.98,22.05,23.58


### Define a function to make a transcript dictionary for a single condition

In [8]:
def transcript_df_to_transcript_dictionary(transcriptomics_df, condition):
    transcript_dict = {}
    for index, row in transcriptomics_df.iterrows():
        transcript_dict[row['YALI0_id']] = row[condition]
    return transcript_dict

glucose_1_transcripts = transcript_df_to_transcript_dictionary(transcriptomics_df, 'glucose_1')
glucose_1_transcripts


{'YALI0A00110g': 4.88,
 'YALI0A00132g': 1896.43,
 'YALI0A00154g': 31.34,
 'YALI0A00176g': 3.67,
 'YALI0A00198g': 0.0,
 'YALI0A00212g': 9.15,
 'YALI0A00264g': 489.68,
 'YALI0A00286g': 244.82,
 'YALI0A00330g': 32.58,
 'YALI0A00352g': 1691.33,
 'YALI0A00369g': 0.81,
 'YALI0A00374g': 2.64,
 'YALI0A00396g': 33.59,
 'YALI0A00418g': 7.0,
 'YALI0A00440g': 137.53,
 'YALI0A00506g': 44.11,
 'YALI0A00528g': 0.0,
 'YALI0A00550g': 4.56,
 'YALI0A00572g': 0.68,
 'YALI0A00594g': 476.9,
 'YALI0A00616g': 40.21,
 'YALI0A00638g': 57.82,
 'YALI0A00660g': 18.78,
 'YALI0A00682g': 0.0,
 'YALI0A00759g': 30.68,
 'YALI0A00781g': 38.97,
 'YALI0A00803g': 145.71,
 'YALI0A00825g': 11.83,
 'YALI0A00847g': 23.73,
 'YALI0A00869g': 0.22,
 'YALI0A00891g': 148.29,
 'YALI0A00913g': 52.82,
 'YALI0A00935g': 2.52,
 'YALI0A00957g': 16.08,
 'YALI0A00979g': 31.73,
 'YALI0A01001g': 43.35,
 'YALI0A01023g': 116.43,
 'YALI0A01045g': 70.65,
 'YALI0A01067g': 142.27,
 'YALI0A01089g': 10.03,
 'YALI0A01111g': 39.55,
 'YALI0A01133g': 74.49

### Calculate E-Flux2 Solutions

In [5]:
%%time

# run E-Flux2 to get genome scale fluxes
eflux2_solution = get_eflux2_solution(model, transcriptomics_df, 'glucose', substrate_uptake_rate=100)

KeyError: 'Growth_Phenol'

In [7]:
%%time

# create dictionary to hold E-Flux2 solutions
eflux2_solutions = {}

# loop over time points and trials
for time in ['T1', 'T2']:
    for trial in ['WT-Glu-R1', 'WT-Glu-R2','WT-Glu-R3']:
                
        # get trial name
        trial_number = trial.split('-R')[1]
        trial_name = f'glucose_eflux2_{time.lower()}_{trial_number}'
        print(trial_name)
        
        # isolate transcriptomics data from a single time point
        if time == 'T1':
            transcript_df = transcript_df_T1[transcript_df_T1['Line Name'] == trial]
        else:
            transcript_df = transcript_df_T2[transcript_df_T2['Line Name'] == trial]
            
        # remove unneeded columns
        transcript_df = transcript_df.filter(['Count', 'Measurement Type'])

        # set index of dataframe to be gene names
        transcript_df = transcript_df.set_index('Measurement Type')
        
        # run E-Flux2 to get genome scale fluxes
        EFlux2_solution = get_EFlux2_solution(model, transcript_df, 'glucose', substrate_uptake_rate=100)
        
        # add E-Flux2 solution to dictionary
        eflux2_solutions[trial_name] = EFlux2_solution

glucose_eflux2_t1_1


NameError: name 'transcript_df_T1' is not defined